# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse Shipments   Customer ShipmentStatus
0  Sep  8 2022  9:31AM  245933        12   HH6720H  Hush Hush       Released
1  Sep  8 2022  9:31AM  245932        12   HH6720G  Hush Hush       Released
2  Sep  8 2022  9:22AM  245931        12  HH-36804  Hush Hush       Released
3  Sep  8 2022  9:22AM  245931        12  HH-36805  Hush Hush       Released
4  Sep  8 2022  9:22AM  245931        12  HH-36806  Hush Hush       Released
5  Sep  8 2022  9:22AM  245931        12  HH-36807  Hush Hush       Released
6  Sep  8 2022  9:22AM  245931        12  HH-36808  Hush Hush       Released
7  Sep  8 2022  9:22AM  245931        12  HH-36809  Hush Hush       Released
8  Sep  8 2022  9:22AM  245931        12  HH-36810  Hush Hush       Released
9  Sep  8 2022  9:22AM  245931        12  HH-36811  Hush Hush       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
45  245929       Released          4
46  245930       Released          4
47  245931       Released         12
48  245932       Released          1
49  245933       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
245929                NaN        NaN       4.0
245930                NaN        NaN       4.0
245931                NaN        NaN      12.0
245932                NaN        NaN       1.0
245933                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
245648                0.0        1.0       0.0
245649                0.0        0.0       1.0
245650                0.0        0.0       1.0
245714               25.0        1.0       0.0
245729                0.0        1.0       0.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
245648                  0          1         0
245649                  0          0         1
245650                  0          0         1
245714                 25          1         0
245729                  0          1         0

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               245648          0          1         0
1               245649          0          0         1
2               245650          0          0         1
3               245714         25          1         0
4               245729          0          1         0

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               245648                   1         
1               245649                            1
2               245650                            1
3               245714        25         1         
4               245729                   1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                 Customer
0   Sep  8 2022  9:31AM  245933        12                Hush Hush
1   Sep  8 2022  9:31AM  245932        12                Hush Hush
2   Sep  8 2022  9:22AM  245931        12                Hush Hush
14  Sep  8 2022  9:19AM  245929        10         Eywa Pharma Inc.
18  Sep  8 2022  9:19AM  245928        10         Eywa Pharma Inc.
37  Sep  8 2022  9:16AM  245930        10           Citieffe, Inc.
41  Sep  8 2022  9:08AM  245927        19  GUSA Granules USA, Inc.
42  Sep  8 2022  9:06AM  245926        10      MedStone Pharma LLC
43  Sep  8 2022  9:01AM  245925        10              Emerginnova
45  Sep  8 2022  8:45AM  245924        10           Eye Pharma Inc

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                 Customer Completed  \
0  Sep  8 2022  9:31AM  245933        12                Hush Hush             
1  Sep  8 2022  9:31AM  245932        12                Hush Hush             
2  Sep  8 2022  9:22AM  245931        12                Hush Hush             
3  Sep  8 2022  9:19AM  245929        10         Eywa Pharma Inc.             
4  Sep  8 2022  9:19AM  245928        10         Eywa Pharma Inc.             
5  Sep  8 2022  9:16AM  245930        10           Citieffe, Inc.             
6  Sep  8 2022  9:08AM  245927        19  GUSA Granules USA, Inc.             
7  Sep  8 2022  9:06AM  245926        10      MedStone Pharma LLC             
8  Sep  8 2022  9:01AM  245925        10              Emerginnova             
9  Sep  8 2022  8:45AM  245924        10           Eye Pharma Inc             

  Executing Released  
0                  1  
1                  1  
2                 12  
3                  4  
4                 19  
5                  4  
6         1           
7                  1  
8                  2  
9                  2

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                 Customer Released  \
0  Sep  8 2022  9:31AM  245933        12                Hush Hush        1   
1  Sep  8 2022  9:31AM  245932        12                Hush Hush        1   
2  Sep  8 2022  9:22AM  245931        12                Hush Hush       12   
3  Sep  8 2022  9:19AM  245929        10         Eywa Pharma Inc.        4   
4  Sep  8 2022  9:19AM  245928        10         Eywa Pharma Inc.       19   
5  Sep  8 2022  9:16AM  245930        10           Citieffe, Inc.        4   
6  Sep  8 2022  9:08AM  245927        19  GUSA Granules USA, Inc.            
7  Sep  8 2022  9:06AM  245926        10      MedStone Pharma LLC        1   
8  Sep  8 2022  9:01AM  245925        10              Emerginnova        2   
9  Sep  8 2022  8:45AM  245924        10           Eye Pharma Inc        2   

  Executing Completed  
0                      
1                      
2                      
3                      
4                      
5                      
6         1            
7                      
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse          Customer Released Executing  \
0  Sep  8 2022  9:31AM  245933        12         Hush Hush        1             
1  Sep  8 2022  9:31AM  245932        12         Hush Hush        1             
2  Sep  8 2022  9:22AM  245931        12         Hush Hush       12             
3  Sep  8 2022  9:19AM  245929        10  Eywa Pharma Inc.        4             
4  Sep  8 2022  9:19AM  245928        10  Eywa Pharma Inc.       19             

  Completed  
0            
1            
2            
3            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse          Customer  Released  \
0  Sep  8 2022  9:31AM  245933        12         Hush Hush       1.0   
1  Sep  8 2022  9:31AM  245932        12         Hush Hush       1.0   
2  Sep  8 2022  9:22AM  245931        12         Hush Hush      12.0   
3  Sep  8 2022  9:19AM  245929        10  Eywa Pharma Inc.       4.0   
4  Sep  8 2022  9:19AM  245928        10  Eywa Pharma Inc.      19.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse          Customer  Released  \
0  Sep  8 2022  9:31AM  245933        12         Hush Hush       1.0   
1  Sep  8 2022  9:31AM  245932        12         Hush Hush       1.0   
2  Sep  8 2022  9:22AM  245931        12         Hush Hush      12.0   
3  Sep  8 2022  9:19AM  245929        10  Eywa Pharma Inc.       4.0   
4  Sep  8 2022  9:19AM  245928        10  Eywa Pharma Inc.      19.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3934476     137.0        2.0       22.0
12          737796      14.0        0.0        0.0
15          737379      64.0        2.0       70.0
16          737666       8.0        0.0        0.0
19          737758       3.0       10.0        0.0
20          983507       8.0       14.0       13.0
21         1965786       4.0        4.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3934476     137.0        2.0       22.0
1        12   737796      14.0        0.0        0.0
2        15   737379      64.0        2.0       70.0
3        16   737666       8.0        0.0        0.0
4        19   737758       3.0       10.0        0.0
5        20   983507       8.0       14.0       13.0
6        21  1965786       4.0        4.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     137.0        2.0       22.0
1        12      14.0        0.0        0.0
2        15      64.0        2.0       70.0
3        16       8.0        0.0        0.0
4        19       3.0       10.0        0.0
5        20       8.0       14.0       13.0
6        21       4.0        4.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  137.0
1        12  Released   14.0
2        15  Released   64.0
3        16  Released    8.0
4        19  Released    3.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10    12    15   16    19    20   21
Status                                            
Completed   22.0   0.0  70.0  0.0   0.0  13.0  0.0
Executing    2.0   0.0   2.0  0.0  10.0  14.0  4.0
Released   137.0  14.0  64.0  8.0   3.0   8.0  4.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10    12    15   16    19    20   21
0          Completed   22.0   0.0  70.0  0.0   0.0  13.0  0.0
1          Executing    2.0   0.0   2.0  0.0  10.0  14.0  4.0
2           Released  137.0  14.0  64.0  8.0   3.0   8.0  4.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10    12    15   16    19    20   21
0  Completed   22.0   0.0  70.0  0.0   0.0  13.0  0.0
1  Executing    2.0   0.0   2.0  0.0  10.0  14.0  4.0
2   Released  137.0  14.0  64.0  8.0   3.0   8.0  4.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()